<h1><center>Простые векторные модели текста</center></h1>

<img src="pipeline_vec.png" alt="pipeline.png" style="width: 400px;"/>

### Задача: классификация твитов по тональности

В этом занятии мы познакомимся с распространенной задачей в анализе текстов: с классификацией текстов на классы.

В рассмотренном тут примере классов будет два: положительный и отрицательный, такую постановку этой задачи обычно называют классификацией по тональности или sentiment analysis.

Классификацию по тональности используют, например, в рекомендательных системах и при анализе отзывов клиентов, чтобы понять, понравилось ли людям кафе, кино, etc.

Более подробно мы рассмотрим данную задачу и познакомимся с более сложными методами её решения в семинаре 3, а здесь разберем простые подходы, основанные на методе мешка слов.

У нас есть [данные постов в твиттере](http://study.mokoron.com/), про из которых каждый указано, как он эмоционально окрашен: положительно или отрицательно. 

**Задача**: построить модель, которая по тексту поста предсказывает его эмоциональную окраску.


Скачиваем данные: [положительные](https://drive.google.com/file/d/1mW_fUtYmRF19AXVySU0gJOIgx0-1EFgD/view?usp=sharing), [отрицательные](https://drive.google.com/file/d/1ZnsFuf-yfO3UEHlIpk7TTqfKkEMdm1EQ/view?usp=sharing).

In [3]:
# если у вас линукс / мак / collab или ещё какая-то среда, в которой работает wget, можно так:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1mW_fUtYmRF19AXVySU0gJOIgx0-1EFgD' -O positive.csv
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1ZnsFuf-yfO3UEHlIpk7TTqfKkEMdm1EQ' -O negative.csv

In [9]:
import pandas as pd
import numpy as np
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', 800)

In [11]:
positive = pd.read_csv('positive.csv', sep=';', usecols=[3], names=['text'])
positive['label'] = ['positive'] * len(positive)
negative = pd.read_csv('negative.csv', sep=';', usecols=[3], names=['text'])
negative['label'] = ['negative'] * len(negative)
df = positive.append(negative)

In [11]:
df.sample(5)

,text,label
16748,"Лес рук, зажигалки, флаги)) #lumen #glavclub #главclub @ ГлавClub http://t.co/YTWzMFnrkt",positive
91284,Влюбиться отменяется. Завтра по плану отличная посиделка с кальяном после работы^_^,positive
97845,@aruslanmager )) она на такие слова ещё и обижается,positive
95348,#юмор ска только 675(( чувакии поднажмите пожааалуйста:33,negative
72398,"однако :D \nузнала о себе\nТы учишь немецкий, живёшь в России... Но тебе пошло бы быть англичанкой.",positive


Воспользуемся функцией для предобработки текста, которую мы написали в прошлом семинаре:

In [16]:
import re
from pymorphy2 import MorphAnalyzer
from functools import lru_cache
from nltk.corpus import stopwords

m = MorphAnalyzer()
regex = re.compile("[А-Яа-яA-z]+")

def words_only(text, regex=regex):
    try:
        return regex.findall(text.lower())
    except:
        return []

In [29]:
@lru_cache(maxsize=128)
def lemmatize_word(token, pymorphy=m):
    return pymorphy.parse(token)[0].normal_form

def lemmatize_text(text):
    return [lemmatize_word(w) for w in text]


mystopwords = stopwords.words('russian') 
def remove_stopwords(lemmas, stopwords = mystopwords):
    return [w for w in lemmas if not w in stopwords and len(w) > 3]

def clean_text(text):
    tokens = words_only(text)
    lemmas = lemmatize_text(tokens)
    
    return ' '.join(remove_stopwords(lemmas))

In [20]:
from multiprocessing import Pool
from tqdm import tqdm

with Pool(4) as p:
    lemmas = list(tqdm(p.imap(clean_text, df['text']), total=len(df)))
    
df['lemmas'] = lemmas
df.sample(5)

100%|██████████| 226834/226834 [01:52<00:00, 2019.78it/s]


,text,label,lemmas
74560,"RT @kostossi: @VechernijUrgant за здоровый образ жизни ! Только каши , льняная подойдет :)",positive,"[kostossi, vechernijurgant, здоровый, образ, жизнь, каша, льняной, подойти]"
6135,))))Я хочу незабываемый Новый год. Много-много снега и близких людей рядом.,positive,"[хотеть, незабываемый, новый, снег, близкие, человек]"
74689,"@KSHN мучают бедную слониху, изверги. Издеваются подлые твари (((((",negative,"[kshn, мучить, бедный, слониха, изверг, издеваться, подлый, тварь]"
51237,Оцарапала гитару медиатором при игре( надеюсь все хорошо будет с ней! Потому что не слабо!,negative,"[оцарапать, гитара, медиатор, игра, надеяться, весь, слабо]"
13634,"Двадцать на двадцать.\n""Программа"" в #kz есть такая.)",positive,"[двадцать, двадцать, программа]"


Разбиваем на train и test:

In [31]:
x_train, x_test, y_train, y_test = train_test_split(df.lemmas, df.label)

## Мешок слов (Bag of Words, BoW)


In [32]:
from sklearn.linear_model import LogisticRegression 
from sklearn.feature_extraction.text import CountVectorizer

... Но сперва пару слов об n-граммах. Что такое n-граммы:

In [23]:
from nltk import ngrams

In [24]:
sent = 'Если б мне платили каждый раз'.split()
list(ngrams(sent, 1)) # униграммы

[('Если',), ('б',), ('мне',), ('платили',), ('каждый',), ('раз',)]

In [25]:
list(ngrams(sent, 2)) # биграммы

[('Если', 'б'),
 ('б', 'мне'),
 ('мне', 'платили'),
 ('платили', 'каждый'),
 ('каждый', 'раз')]

In [26]:
list(ngrams(sent, 3)) # триграммы

[('Если', 'б', 'мне'),
 ('б', 'мне', 'платили'),
 ('мне', 'платили', 'каждый'),
 ('платили', 'каждый', 'раз')]

In [27]:
list(ngrams(sent, 5)) # ... пентаграммы?

[('Если', 'б', 'мне', 'платили', 'каждый'),
 ('б', 'мне', 'платили', 'каждый', 'раз')]

Итак, мы хотим преобразовать наши обработанные данные в вектора с помощью мешка слов. Мешок слов можно строить как для отдельных слов (лемм в нашем случае), так и для n-грамм, и это может улучшать качество. 

Объект `CountVectorizer` делает простую вещь:
* строит для каждого документа (каждой пришедшей ему строки) вектор размерности `n`, где `n` -- количество слов или n-грам во всём корпусе
* заполняет каждый i-тый элемент количеством вхождений слова в данный документ

In [33]:
vec = CountVectorizer(ngram_range=(1, 1)) # строим BoW для слов
bow = vec.fit_transform(x_train) 

ngram_range отвечает за то, какие n-граммы мы используем в качестве признаков:<br/>
ngram_range=(1, 1) -- униграммы<br/>
ngram_range=(3, 3) -- триграммы<br/>
ngram_range=(1, 3) -- униграммы, биграммы и триграммы.

В vec.vocabulary_ лежит словарь: соответствие слов и их индексов в словаре:

In [34]:
list(vec.vocabulary_.items())[:10]

[('докторша', 108072),
 ('говорить', 104475),
 ('поликлинник', 140005),
 ('милый', 125025),
 ('красивый', 119704),
 ('урурурур', 160311),
 ('мило', 124998),
 ('sopli', 73121),
 ('back', 9457),
 ('средневековье', 153620)]

In [36]:
bow[0]

<1x169087 sparse matrix of type '<class 'numpy.int64'>'
	with 7 stored elements in Compressed Sparse Row format>

Теперь у нас есть вектора, на которых можно обучать модели! 

In [37]:
clf = LogisticRegression(random_state=42, max_iter=500)
clf.fit(bow, y_train)

LogisticRegression(max_iter=500, random_state=42)

Посмотрим на качество классификации на тестовой выборке. Для этого выведем classification_report из модуля [sklearn.metrics](https://scikit-learn.org/stable/modules/classes.html#sklearn-metrics-metrics)

В качестве целевой метрики качества будем рассматривать macro average f1-score.

In [38]:
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       0.74      0.73      0.73     28300
    positive       0.73      0.74      0.74     28409

    accuracy                           0.74     56709
   macro avg       0.74      0.74      0.74     56709
weighted avg       0.74      0.74      0.74     56709



Попробуем сделать то же самое для триграмм:

In [39]:
vec = CountVectorizer(ngram_range=(3, 3))
bow = vec.fit_transform(x_train)
clf = LogisticRegression(random_state=42, max_iter = 300)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       0.97      0.53      0.68     51145
    positive       0.16      0.84      0.27      5564

    accuracy                           0.56     56709
   macro avg       0.56      0.68      0.48     56709
weighted avg       0.89      0.56      0.64     56709



Видим, что качество существенно хуже. Ниже мы поймем, почему это так.

## TF-IDF векторизация

`TfidfVectorizer` делает то же, что и `CountVectorizer`, но в качестве значений – tf-idf каждого слова.

Как считается tf-idf:

TF (term frequency) – относительная частотность слова в документе:
$$ TF(t,d) = \frac{n_t}{\sum_k n_k} $$

`t` -- слово (term), `d` -- документ, $n_t$ -- количество вхождений слова, $n_k$ -- количество вхождений остальных слов

IDF (inverse document frequency) – обратная частота документов, в которых есть это слово:
$$ IDF(t, D) = \mbox{log} \frac{|D|}{|{d : t \in d}|} $$

`t` -- слово (term), `D` -- коллекция документов

Перемножаем их:
$$TFIDF(t,d,D) = TF(t,d) \times IDF(i, D)$$

Ключевая идея этого подхода – если слово часто встречается в одном документе, но в целом по корпусу встречается в небольшом 
количестве документов, у него высокий TF-IDF.

In [40]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [42]:
vec = TfidfVectorizer(ngram_range=(1, 1))
bow = vec.fit_transform(x_train)
clf = LogisticRegression(random_state=42, max_iter = 500)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       0.70      0.74      0.72     26124
    positive       0.77      0.72      0.74     30585

    accuracy                           0.73     56709
   macro avg       0.73      0.73      0.73     56709
weighted avg       0.73      0.73      0.73     56709



В этот раз получилось хуже, чем с помощью простого CountVectorizer, то есть использование tf-idf не дало улучшений в качестве. 

## О важности эксплоративного анализа

Иногда в ходе стандартного препроцессинга теряются важные признаки. Посмотрим, что будет если не убирать пунктуацию?

In [44]:
df.sample()

,text,label,lemmas
106338,"Короче, удивила Москва меня ! Да так-то тут нормально)",positive,короче удивить москва нормальный


In [46]:
df['new_lemmas'] = df.text.apply(lambda x: x.lower())
df.sample(3)

,text,label,lemmas,new_lemmas
91634,@AdolfNastya у нас было -20 \nчто ты знаешь о морозе?:D,positive,adolfnastya знаешь мороз,@adolfnastya у нас было -20 \nчто ты знаешь о морозе?:d
35566,"@Florida_1995 @Shutova4Real значит, курочки, да? это ты на наши умственные способности намекаешь? :D хорошо, что хоть любимые) ахаха",positive,florida_ shutova real значит курочка умственный способность намекать любимый ахах,"@florida_1995 @shutova4real значит, курочки, да? это ты на наши умственные способности намекаешь? :d хорошо, что хоть любимые) ахаха"
81916,ты загодала желание?- нет ! - почему?- мне не надо загадывать ты уже со мной!!!!))),positive,загодать желание почему загадывать,ты загодала желание?- нет ! - почему?- мне не надо загадывать ты уже со мной!!!!)))


In [47]:
x_train, x_test, y_train, y_test = train_test_split(df.new_lemmas, df.label)

In [51]:
from nltk import word_tokenize

vec = TfidfVectorizer(ngram_range=(1, 1), tokenizer=word_tokenize)
bow = vec.fit_transform(x_train)
clf = LogisticRegression(random_state=42, max_iter = 300)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       1.00      1.00      1.00     27825
    positive       1.00      1.00      1.00     28884

    accuracy                           1.00     56709
   macro avg       1.00      1.00      1.00     56709
weighted avg       1.00      1.00      1.00     56709



Как можно видеть, если оставить пунктуацию, то все метрики равны 1. 

In [59]:
len(vec.vocabulary_), len(clf.coef_[0])

(259979, 259979)

In [60]:
importances = list(zip(vec.vocabulary_, clf.coef_[0]))
importances[0]

('@', 0.11768651169721642)

In [72]:
sorted_importances = sorted(importances, key = lambda x: -x[1])
sorted_importances[:10]

[('есть', 58.39173024498836),
 ('foxellanseva', 26.9157592150693),
 ('чуваку', 10.576893100095987),
 ('theirinali', 9.096114872867306),
 ('рта', 7.8846010394951085),
 ('событий', 7.811647650160388),
 (')', 7.102713299790951),
 ('лжецы', 6.178342975649683),
 ('освободились', 4.904776606595675),
 ('сформулировать', 3.0981650553247335)]

Посмотрим, как один из наиболее значительных токенов справится с классификацией безо всякого машинного обучения:

In [73]:
cool_token = ')'
pred = ['positive' if cool_token in tweet else 'negative' for tweet in x_test]
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       1.00      0.85      0.92     32706
    positive       0.83      1.00      0.91     24003

    accuracy                           0.91     56709
   macro avg       0.92      0.93      0.91     56709
weighted avg       0.93      0.91      0.92     56709



Можно видеть, что это уже позволяет достаточно хорошо классифицировать тексты.

## Символьные n-граммы

Теперь в качестве признаком используем, например, униграммы символов:

In [74]:
vec = CountVectorizer(analyzer='char', ngram_range=(1, 1))
bow = vec.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

/Library/Python/3.7/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


              precision    recall  f1-score   support

    negative       0.99      1.00      0.99     27742
    positive       1.00      0.99      0.99     28967

    accuracy                           0.99     56709
   macro avg       0.99      0.99      0.99     56709
weighted avg       0.99      0.99      0.99     56709



Таким образом, становится понятно, почему на этих данных качество классификации 1. Так или иначе, на символах классифицировать тоже можно.

Ещё одна замечательная особенность символьных признаков: токенизация и лемматизация не нужна, можно использовать такой подход для языков, у которых нет готовых анализаторов.

## Итоги

 На этом занятии мы
* познакомились с задачей бинарной классификации текстов.

* научились строить простые признаки на основе метода "мешка слов" с помощью библиотеки sklearn: CountVectorizer и TfidfVectorizer.

* использовали для классификации линейную модель логистической регрессии.

* поняли, что многое зависит от подхода к предобработки текста и от признаков, которые используются в модели.

* увидели, что в некоторых задачах важно использование каждого символа из текста, в том числе пунктуации.

На следующих занятиях мы рассмотрим более сложные модели построения признаков и классификации текстов.